# 配置环境

在运行之前需要安装所需的python包。建议使用anaconda创建一个新的虚拟环境，具体方法为打开Anaconda navigator，创建环境，环境名称可以命名为ByteBites。

激活环境，确保在vscode的资源管理器中打开ByteBites文件夹。在vscode的左侧python扩展中，在global environments，选中刚才安装的环境，点击open in terminal，在打开的命令行中输入以下命令安装所需的包：
```bash
pip install -r italy/requirements.txt
```

In [9]:
import pandas as pd
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,)
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough
from langchain_community.document_loaders.dataframe import DataFrameLoader
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

DATASET_PATH = "..\\Amap-results_NJU-Gulou-3000m.csv"

# 导入CSV数据库

以下代码从CSV文件导入餐厅数据，并生成一个新的文档对象documents。这个功能由DataFrameLoader提供。

### 数据结构说明

documents可以理解为一个list表格，每一行是包含两个字段的dict，两个字段分别为：  
- `page_content`：值为字符串。将餐厅的所有信息整合在一个字符串中，用于喂给大模型。
- `metadata`：值为dict。储存准备用于初过滤的硬性指标值，如经纬度、营业时间等。 

### 示例

假设原始csv文件如下：
```csv
name,address,location,type,tel,cost,rating,opentime_today,opentime_week,tag
巴蜀鱼花(南大店),湖南路街道汉口路30号,"118.779220,32.053685",餐饮服务;中餐厅;火锅店,15380870767,,4.4,09:00-21:00,周一至周日 09:00-21:00,
陕老顺肉夹馍,汉口路30号,"118.779105,32.053716",餐饮服务;餐饮相关场所;餐饮相关,15924140124,,4.4,10:00-21:00,周一至周日 10:00-21:00,肉夹馍
```

导入后的结果为：
```python
documents = [
    ..., # 第一行略
    {
        "page_content": "name=陕老顺肉夹馍\naddress=汉口路30号\nlocation=118.779105,32.053716\ntype=餐饮服务;餐饮相关场所;餐饮相关\ntag=肉夹馍\nrating=4.4\nopentime_today=10:00-21:00\nopentime_week=周一至周日 10:00-21:00\ntel=15924140124",
        "metadata": {
            "location": "118.779105,32.053716",
            "opentime_week": "周一至周日 10:00-21:00"
        }
    }
]
```

### **注意**

- 读取店铺数据的代码更改，导致csv文件中字段发生变化时：def content_func 中 python content_fields 的值要随之更改。
- 想作为硬性指标的字段发生变化时：调用部分 metadata_fields 中的值要更改。

In [10]:
def get_documents(content_func=lambda row: row['name'] + '\n' + row['tag'],  # 用于导入page_content的函数。如果调用时没传这个参数，默认是将每一行的name和tag拼接起来。
                  # source_func=lambda row: row['address'],  # 后面没用到这个函数。用于导入source的函数。默认是将address作为文档来源。
                  metadata_fields=[]):  # 导入metadata（一个dict）时打算加入进去的字段列表，默认为空。

    # 加载数据库，读取指定路径的 CSV 文件
    dataset_df = pd.read_csv(DATASET_PATH)
    dataset_df.drop_duplicates(inplace=True) # 删除重复数据，确保数据唯一性

    # 对原有数据分别用函数content_func和source_func处理，添加新的列 'page_content' 和'source'，这两列的值由两个函数得到
    dataset_df['page_content'] = dataset_df.apply(content_func, axis=1)
    # dataset_df['source'] = dataset_df.apply(source_func, axis=1)

    # 将 'page_content' 字段添加到metadata_fields中
    metadata_fields = list(set(metadata_fields + ['page_content']))

    # 使用 DataFrameLoader 生成一个新文档对象。 'page_content'列作为文档内容，其他所有列都作为metadata的内容
    loader = DataFrameLoader(dataset_df[metadata_fields], page_content_column='page_content')
    return loader.load()

def content_func(row) -> str: # 定义content_func函数，用于把每家店的所有信息拼到一起，返回成一个字符串。中间用换行符隔开。
  content_fields = ["name",
                    "address",
                    # "location", 
                    "type",
                    "tag",
                    "cost",
                    "rating",
                    "opentime_today",
                    "opentime_week", 
                    # "tel"
                    ]
  return '\n'.join(f"{key}={row[key]}" for key in content_fields if pd.notna(row[key]))

# 调用
metadata_fields = ["location", "opentime_week"]
documents = get_documents(content_func, metadata_fields=metadata_fields)

# 展示
print(documents[1].page_content)
print(documents[1].metadata)

In [11]:
# from dotenv import load_dotenv
# from langchain_openai import OpenAIEmbeddings
# import os

# #加载环境变量
# load_dotenv()
# #配置嵌入模型
# EMBEDDING_MODEL_NAME = "text-embedding-3-small"  
# embedding_model = OpenAIEmbeddings(
#     model=EMBEDDING_MODEL_NAME,
#     openai_api_key=os.getenv("OPENAI_API_KEY")
# )

# 配置嵌入vecterbase所用的模型

用的是huggingface上面的某个轻量级开源模型。

In [12]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import os
import numpy as np

# 初始化 HuggingFaceEmbeddings 模型
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",  # 轻量级模型
    model_kwargs={"device": "cpu"},  # 如果电脑上安装了英伟达显卡，可改成 "cuda" 来加速
    encode_kwargs={"normalize_embeddings": True})  # 归一化

# 生成查询的嵌入向量
result = embedding_model.embed_query("查询向量示例")

# 将结果转换为 numpy 数组并打印信息
array = np.array(result)
print(f"embedding shape: {array.shape}\nembedding norm: {np.linalg.norm(array, ord=2)}")

# FAISS数据库

在代码 FAISS_REVIEWS_PATH_COSINE = "faiss_index_cosine" 中，"faiss_index_cosine" 是 FAISS 向量索引的本地存储路径，保存的是经过向量化处理后的文档索引数据。以下是详细解释：

存储的具体内容
当调用 vector_db.save_local(FAISS_REVIEWS_PATH_COSINE) 时，会在该路径下生成以下文件：

index.faiss
二进制文件，存储向量索引的核心数据（包括向量数据、索引结构等）。

index.pkl（可选）
存储元数据（如文档的原始文本、ID等，需通过 LangChain 额外配置）。

这些文件共同构成一个完整的可复用的向量数据库。

In [13]:
FAISS_REVIEWS_PATH_COSINE = "faiss_index_cosine" # 向量库存储路径
FAISS_INDEX_NAME = "index" # 向量库索引名称
FAISS_DISTANCE_STRATEGY_COSINE = "COSINE_DISTANCE" # 向量库距离计算策略

# 用于根据csv数据生成向量库的函数。documents就是前面csv数据的导入结果。embedding_model就是上面定义的嵌入模型。
def get_vector_database(documents, embedding_model, distance_strategy):

  vector_database = FAISS.from_documents(
      documents, embedding_model,
      distance_strategy= distance_strategy
      )
  return vector_database

# 嵌入向量库。分批次处理文档，加入了等待时间，避免API限制（现在的版本是把向量库保存在本地，没有对应限制）。
import time
doclen = len(documents) # 这里的长度指的是前面读的数据的行数。
for batch in range(doclen//100 + 1): # 将每个店铺的信息独立转换为一个向量，且每次并行处理 100 个店铺的向量
    docs = documents[batch*100:(batch+1)*100]
    if batch == 0:
        vector_db = get_vector_database(docs, embedding_model, FAISS_DISTANCE_STRATEGY_COSINE)
    else:
        vector_db.merge_from(get_vector_database(docs, embedding_model, FAISS_DISTANCE_STRATEGY_COSINE))
    time.sleep(10) # 每次处理完100条数据，休眠10秒，防止api限制。
    
#储存并加载向量库
vector_db.save_local(folder_path=FAISS_REVIEWS_PATH_COSINE, index_name=FAISS_INDEX_NAME)
vector_db = FAISS.load_local(folder_path=FAISS_REVIEWS_PATH_COSINE,
                             embeddings=embedding_model,
                             index_name=FAISS_INDEX_NAME,
                             allow_dangerous_deserialization=True) # 允许反序列化

### 验证效果

In [ ]:
docs = vector_db.similarity_search("馄饨，汉口路", k = 5)
for doc in docs:
    print(doc, end="\n\n")

In [15]:
docs = vector_db.similarity_search("披萨", k = 5)
for doc in docs:
    print(doc, end="\n\n")

# 配置大语言模型
需要在ByteBites目录下创建配置文件.env，里面添加大语言模型的配置信息。密钥要自己申请。代码支持的模型包括：OpenAI、Deepseek、通义千问等。

.env文件形如：

```python
# OpenAI
OPENAI_API_KEY = 
OPENAI_MODEL_NAME = "gpt-4o-mini"

# Deepseek
DEEPSEEK_API_KEY = 
DEEPSEEK_BASE_URL = 'https://api.deepseek.com'
DEEPSEEK_MODEL_V3 = 'deepseek-chat'
DEEPSEEK_MODEL_R1 = 'deepseek-reasoner'
```

In [26]:
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
import getpass

# 载入本地 .env 文件
load_dotenv()

# 设置 DeepSeek API KEY
if not os.environ.get("DEEPSEEK_API_KEY"):
    os.environ["DEEPSEEK_API_KEY"] = getpass.getpass("Enter API key for DeepSeek-AI: ")

# 初始化 LangChain 的 ChatOpenAI（适配 DeepSeek）
llm = ChatOpenAI(
    openai_api_key=os.environ["DEEPSEEK_API_KEY"],
    openai_api_base="https://api.deepseek.com/v1",
    model_name="deepseek-chat",  # 或者 deepseek-v3，具体根据官方命名
    temperature=0.7
)

# 示例对话调用
response = llm([HumanMessage(content="你好，请用一句话介绍你自己")])
print(response.content)

# 用LangChain包设置和大模型交互的工作流

In [27]:
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables.history import RunnableWithMessageHistory

# ========== 需编辑部分 ==============================
system_prompt_template = """
# 你的角色

你是一个餐厅推荐助手。你的工作是使用数据库中的餐馆详细信息来为用户推荐最佳就餐地点。

# 规则

你只能根据数据库中的信息回答用户问题。
如果你发现数据库中有与主题无关的信息，请忽略。
如果你发现用户问题与就餐无关，请告诉用户你只能回答与餐厅相关的问题。
如果你看了数据库后还是不知道答案，就说你不知道。

# 数据库的格式

每家餐馆的信息包括：
name餐馆名字
address地址
type餐馆类型
tag餐馆标签
rating用户评分（越高越好）
opentime_today每天营业时间
opentime_week星期几营业

# 数据库内容

{context}
"""

human_prompt_template = """
{question}
"""
# =====================================================

# 使用langchain自带的prompts模板编辑器来编辑prompt格式
system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=system_prompt_template
    )
)
human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question"], template=human_prompt_template)
)
messages = [system_prompt, human_prompt]
total_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question", "chat_history"],  # 加入 chat_history
    messages=messages
)

# 配置RAG的初步检索器。这里是利用上文中配好的vectorbase来检索和用户问题最相关的前20条餐厅信息
reviews_retriever = vector_db.as_retriever(search_kwargs={'k': 20,})

# 初始化对话记忆
memory = ConversationBufferMemory(
    memory_key="chat_history",  # 用于存储对话历史
    return_messages=True        # 返回消息对象
)

# 配置和大模型交互的完整chatbot，加入对话历史
review_chain = (
    {
        "context": reviews_retriever,
        "question": RunnablePassthrough(),
        "chat_history": RunnablePassthrough()  # 传递对话历史
    }
    | total_prompt_template
    | llm
    | StrOutputParser()
)

# 包装带有对话历史的链
chain_with_history = RunnableWithMessageHistory(
    review_chain,
    lambda session_id: memory.chat_memory,  # 使用 memory.chat_memory 管理历史
    input_messages_key="question",
    history_messages_key="chat_history",
)



In [29]:
question = """我想吃肯德基"""
print(review_chain.invoke(question))

## 多轮对话